In [5]:
from astropy.io import fits
import numpy as np
from astropy.table import Table
import sys
sys.path.append('/Users/rociokiman/Documents/Gaia-Cupid/ActivityAgeRelation/banyan_sigma')
from banyan_sigma import banyan_sigma
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord  # High-level coordinates
import astropy.units as u

# Members from the literature

In [6]:
mg = fits.open('Catalogs/mactivity_movinggroups_gaia.fits')

In [7]:
mg_ref = Table.read('Catalogs/moving_groups_ref.csv',format='csv')
source_ref = Table.read('Catalogs/source_ref.csv',format='csv')

In [8]:
ra = np.array([float(x) for x in mg[1].data['ra_x']])
ra_err = np.array([float(x) for x in mg[1].data['ra_error']])
dec = np.array([float(x) for x in mg[1].data['dec_x']])
pmra = np.array([float(x) for x in mg[1].data['pmra']])
pmra_error = np.array([float(x) for x in mg[1].data['pmra_error']])
pmdec = np.array([float(x) for x in mg[1].data['pmdec']])
pmdec_error = np.array([float(x) for x in mg[1].data['pmdec_error']])
parallax = np.array([float(x) for x in mg[1].data['parallax']])
parallax_error = np.array([float(x) for x in mg[1].data['parallax_error']])
rv = np.array([float(x) for x in mg[1].data['radial_velocity']])
rv_error = np.array([float(x) for x in mg[1].data['radial_velocity_error']])

In [9]:
len(ra)

4599

In [10]:
mask_nan = (~np.isnan(ra+dec+pmra+pmra_error+pmdec+pmdec_error+parallax+parallax_error) 
           * (parallax/parallax_error > 8))

/Users/rociokiman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [11]:
group = np.array(mg[1].data['group_name'])[mask_nan]

In [12]:
len(ra[mask_nan])

4373

In [13]:
OUTPUT_STRUCTURE1 = banyan_sigma(ra=ra[mask_nan], dec=dec[mask_nan], pmra=pmra[mask_nan], pmdec=pmdec[mask_nan], 
                                epmra=pmra_error[mask_nan], epmdec=pmdec_error[mask_nan],
                                rv=rv[mask_nan], erv=rv_error[mask_nan], 
                                plx=parallax[mask_nan], eplx=parallax_error[mask_nan])

In [14]:
prob_ya = np.array(OUTPUT_STRUCTURE1['YA_PROB']).reshape(len(OUTPUT_STRUCTURE1['YA_PROB']),)
best_ya = np.array(OUTPUT_STRUCTURE1['BEST_YA']).reshape(len(OUTPUT_STRUCTURE1['BEST_YA']),)

In [15]:
highprob = prob_ya>0.8


In [16]:
len(best_ya[highprob])

1645

In [17]:
mg_table = Table.read('Catalogs/mactivity_movinggroups_gaia.fits')
mg_ref = Table.read('Catalogs/moving_groups_ref.csv')

In [18]:
highprob_mask = highprob.reshape(len(highprob),)

# Praesepe Members

In [19]:
source_id_praesepe = np.loadtxt('Catalogs/source_id_pra.txt')

In [20]:
bf_pra = np.array([True if x in source_id_praesepe else False for x in mg[1].data['source_id']])

In [21]:
def remove(string): 
    return string.replace(" ", "")

In [22]:
for i in range(len(mg_table['group_name'])):
    if(remove(mg_table['group_name'][i])=='PRA' and bf_pra[i]):
        mg_table['group_name'][i] = 'PRA'
        mg_table['group_num'][i] = mg_ref['group_num'][mg_ref['name']=='PRA'][0]
        mg_table['age'][i] = mg_ref['age'][mg_ref['name']=='PRA'][0]
        mg_table['age_error'][i] = mg_ref['age_error'][mg_ref['name']=='PRA'][0]

# Literature checked with Banyan + Praesepe

In [23]:
mask_all = np.logical_or(highprob_mask,bf_pra[mask_nan])

In [24]:
mg_table_mg = mg_table[mask_nan][mask_all]

In [25]:
mg_table_mg['ya_prob'] = prob_ya[mask_all]
mg_table_mg['best_ya'] = np.array([str(x) for x in best_ya[mask_all]])

In [26]:
len(mg_table_mg)

1670

# Correcting groups that don't agree

In [27]:
for i in range(len(mg_table_mg)):
    if(mg_table_mg['group_name'][i]!=mg_table_mg['best_ya'][i]):
        if(mg_table_mg['group_name'][i]!='PRA'):
            mask = mg_table_mg['best_ya'][i] == mg_ref['name']
            mg_table_mg['group_name'][i] = mg_ref['name'][mask][0]
            mg_table_mg['group_num'][i] = mg_ref['group_num'][mask][0]
            mg_table_mg['age'][i] = mg_ref['age'][mask][0]
            mg_table_mg['age_error'][i] = mg_ref['age_error'][mask][0]
        elif(mg_table_mg['group_name'][i]=='PRA'):
            mg_table_mg['best_ya'][i] = 'PRA'
            mg_table_mg['ya_prob'][i] = np.nan

# Members from MLSDSS-GaiaDR2

In [28]:
catalog = Table.read('../Gaia-Cupid/Catalogs/MLSDSS-GaiaDR2.fits')

In [29]:
mask_nan2 = (~np.isnan(catalog['RA']+catalog['DEC']+catalog['PMRA']+catalog['PMDEC']
                     +catalog['PMRA_ERR']+catalog['PMDEC_ERR']+catalog['RV']
                     +catalog['RV_ERR']+catalog['parallax']+catalog['parallax_error'])
            * (catalog['GOODMATCH'] == 1) 
            * (catalog['parallax']/catalog['parallax_error']>8))

/Users/rociokiman/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


In [30]:
OUTPUT_STRUCTURE2 = banyan_sigma(ra=catalog['RA'][mask_nan2], dec=catalog['DEC'][mask_nan2], 
                                pmra=catalog['PMRA'][mask_nan2], pmdec=catalog['PMDEC'][mask_nan2], 
                                epmra=catalog['PMRA_ERR'][mask_nan2], 
                                epmdec=catalog['PMDEC_ERR'][mask_nan2],
                                rv=catalog['RV'][mask_nan2], erv=catalog['RV_ERR'][mask_nan2], 
                                plx=catalog['parallax'][mask_nan2], 
                                eplx=catalog['parallax_error'][mask_nan2])

In [31]:
prob_ya = np.array(OUTPUT_STRUCTURE2['YA_PROB']).reshape(len(OUTPUT_STRUCTURE2['YA_PROB']),)
best_ya = np.array(OUTPUT_STRUCTURE2['BEST_YA']).reshape(len(OUTPUT_STRUCTURE2['BEST_YA']),)

In [32]:
highprob = prob_ya>0.8

In [33]:
len(best_ya[highprob])

25

In [34]:
catalog_mg = catalog[mask_nan2][highprob]

In [35]:
catalog_mg['best_ya'] = best_ya[highprob]
catalog_mg['ya_prob'] = prob_ya[highprob]

In [36]:
age = []
age_error = []
group_name = []
group_num = []
for i in range(len(catalog_mg['best_ya'])):
    mask = catalog_mg['best_ya'][i] == mg_ref['name']
    age.append(mg_ref['age'][mask][0])
    age_error.append(mg_ref['age_error'][mask][0])
    group_name.append(mg_ref['name'][mask][0])
    group_num.append(mg_ref['group_num'][mask][0])

source_num = np.ones(len(catalog_mg['ya_prob']))*source_ref['source_num'][source_ref['source_ref']=='Kiman 2019'][0]
source_ref = np.array(['Kiman 2019' for x in range(len(catalog_mg['ya_prob']))])
    
age,age_error,group_name,group_num = np.array(age),np.array(age_error),np.array(group_name),np.array(group_num)

# Compiling full sample of M dwarfs from moving groups

In [37]:
fullsample = Table()

In [38]:
fullsample['ra'] = np.concatenate((mg_table_mg['ra'],catalog_mg['RA']))
fullsample['dec'] = np.concatenate((mg_table_mg['dec'],catalog_mg['DEC']))
fullsample['ra_gaia'] = np.concatenate((mg_table_mg['ra_x'],catalog_mg['RA']))
fullsample['dec_gaia'] = np.concatenate((mg_table_mg['dec_x'],catalog_mg['DEC']))
fullsample['spt'] = np.concatenate((mg_table_mg['spt'],catalog_mg['SPT']))
fullsample['phot_g_mean_mag'] = np.concatenate((mg_table_mg['phot_g_mean_mag'],catalog_mg['phot_g_mean_mag']))
fullsample['phot_rp_mean_mag'] = np.concatenate((mg_table_mg['phot_rp_mean_mag'],catalog_mg['phot_rp_mean_mag']))
fullsample['phot_bp_mean_mag'] = np.concatenate((mg_table_mg['phot_bp_mean_mag'],catalog_mg['phot_bp_mean_mag']))
fullsample['parallax'] = np.concatenate((mg_table_mg['parallax'],catalog_mg['parallax']))
fullsample['parallax_error'] = np.concatenate((mg_table_mg['parallax_error'],catalog_mg['parallax_error']))
fullsample['ewha'] = np.concatenate((mg_table_mg['ewha'],catalog_mg['EWHA']))
fullsample['ewha_error'] = np.concatenate((mg_table_mg['ewha_error'],catalog_mg['EWHA_ERR']))
fullsample['lhalbol'] = np.concatenate((mg_table_mg['lhalbol'],catalog_mg['LHALBOL']))
fullsample['lhalbol_error'] = np.concatenate((mg_table_mg['lhalbol_error'],catalog_mg['LHALBOL_ERR']))
fullsample['age'] = np.concatenate((mg_table_mg['age'],age))
fullsample['age_error'] = np.concatenate((mg_table_mg['age_error'],age_error))
fullsample['group_num'] = np.concatenate((mg_table_mg['group_num'],group_num))
fullsample['group_name'] = np.concatenate((mg_table_mg['group_name'],group_name))
fullsample['ya_prob'] = np.concatenate((mg_table_mg['ya_prob'],catalog_mg['ya_prob']))
fullsample['source_num'] = np.concatenate((mg_table_mg['source_num'],source_num))
fullsample['source_ref'] = np.concatenate((mg_table_mg['source_ref'],source_ref))

# Identify repeated stars

In [51]:
A = [[1,2,3,4],[9,1,2]]

In [62]:
any(1 in sublist for sublist in A)

True

In [64]:
N = len(fullsample['ra'])
same_star = []
for i in range(N):
    if(any(i in sublist for sublist in same_star)): 
        continue
    else:
        same_star_i = []
        same_star_i.append(i)
        c1 = SkyCoord(ra=fullsample['ra'][i]*u.deg, dec=fullsample['dec'][i]*u.deg)
        for j in np.arange(i+1,N):
            c2 = SkyCoord(ra=fullsample['ra'][j]*u.deg, dec=fullsample['dec'][j]*u.deg)
            if(c1.separation(c2).arcsec < 3):
                same_star_i.append(j)
                print(c1,c2)
                same_star.append(same_star_i)

<SkyCoord (ICRS): (ra, dec) in deg
    (57.4302917, 24.3179667)> <SkyCoord (ICRS): (ra, dec) in deg
    (57.43020833, 24.31797222)>
<SkyCoord (ICRS): (ra, dec) in deg
    (57.4302917, 24.3179667)> <SkyCoord (ICRS): (ra, dec) in deg
    (57.430312, 24.317915)>
<SkyCoord (ICRS): (ra, dec) in deg
    (59.809125, 22.04385)> <SkyCoord (ICRS): (ra, dec) in deg
    (59.809125, 22.043849)>
<SkyCoord (ICRS): (ra, dec) in deg
    (59.860875, 23.3854528)> <SkyCoord (ICRS): (ra, dec) in deg
    (59.860875, 23.385453)>
<SkyCoord (ICRS): (ra, dec) in deg
    (60.0649167, 19.4101556)> <SkyCoord (ICRS): (ra, dec) in deg
    (60.064917, 19.410155)>
<SkyCoord (ICRS): (ra, dec) in deg
    (60.28125, 33.1961083)> <SkyCoord (ICRS): (ra, dec) in deg
    (60.281253, 33.196109)>
<SkyCoord (ICRS): (ra, dec) in deg
    (60.9123333, 19.4550944)> <SkyCoord (ICRS): (ra, dec) in deg
    (60.912336, 19.455095)>
<SkyCoord (ICRS): (ra, dec) in deg
    (61.053375, 18.9957111)> <SkyCoord (ICRS): (ra, dec) in deg
    (61

<SkyCoord (ICRS): (ra, dec) in deg
    (6.95929167, -32.55166667)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.9593067, -32.5517508)>
<SkyCoord (ICRS): (ra, dec) in deg
    (6.95929167, -32.55166667)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.9592917, -32.5516667)>
<SkyCoord (ICRS): (ra, dec) in deg
    (6.95929167, -32.55166667)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.95933333, -32.55169444)>
<SkyCoord (ICRS): (ra, dec) in deg
    (6.95929167, -32.55166667)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.95933333, -32.55169444)>
<SkyCoord (ICRS): (ra, dec) in deg
    (6.95929167, -32.55166667)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.95929167, -32.55166667)>
<SkyCoord (ICRS): (ra, dec) in deg
    (6.95979167, -32.55661111)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.9598075, -32.5567025)>
<SkyCoord (ICRS): (ra, dec) in deg
    (6.95979167, -32.55661111)> <SkyCoord (ICRS): (ra, dec) in deg
    (6.95979167, -32.55661111)>
<SkyCoord (ICRS): (ra, dec) in deg
    (83.49920833, -2.35902778)> <

<SkyCoord (ICRS): (ra, dec) in deg
    (40.44708333, -52.99183333)> <SkyCoord (ICRS): (ra, dec) in deg
    (40.4470833, -52.9918333)>
<SkyCoord (ICRS): (ra, dec) in deg
    (40.44708333, -52.99183333)> <SkyCoord (ICRS): (ra, dec) in deg
    (40.44708333, -52.99194444)>
<SkyCoord (ICRS): (ra, dec) in deg
    (40.44708333, -52.99183333)> <SkyCoord (ICRS): (ra, dec) in deg
    (40.447106, -52.9918457)>
<SkyCoord (ICRS): (ra, dec) in deg
    (40.44708333, -52.99183333)> <SkyCoord (ICRS): (ra, dec) in deg
    (40.44708333, -52.99183333)>
<SkyCoord (ICRS): (ra, dec) in deg
    (40.63754167, -57.66019444)> <SkyCoord (ICRS): (ra, dec) in deg
    (40.6375417, -57.6601944)>
<SkyCoord (ICRS): (ra, dec) in deg
    (40.63754167, -57.66019444)> <SkyCoord (ICRS): (ra, dec) in deg
    (40.6376064, -57.6602306)>
<SkyCoord (ICRS): (ra, dec) in deg
    (49.786, -35.11672222)> <SkyCoord (ICRS): (ra, dec) in deg
    (49.786, -35.1167222)>
<SkyCoord (ICRS): (ra, dec) in deg
    (49.786, -35.11672222)> <SkyC

<SkyCoord (ICRS): (ra, dec) in deg
    (311.74145833, -2.99222222)> <SkyCoord (ICRS): (ra, dec) in deg
    (311.74145833, -2.99222222)>
<SkyCoord (ICRS): (ra, dec) in deg
    (326.67841667, -85.71794444)> <SkyCoord (ICRS): (ra, dec) in deg
    (326.67841667, -85.71794444)>
<SkyCoord (ICRS): (ra, dec) in deg
    (353.00075, -39.29355556)> <SkyCoord (ICRS): (ra, dec) in deg
    (353.00075, -39.29355556)>
<SkyCoord (ICRS): (ra, dec) in deg
    (39.21545833, -52.051)> <SkyCoord (ICRS): (ra, dec) in deg
    (39.2154231, -52.0510003)>
<SkyCoord (ICRS): (ra, dec) in deg
    (39.21545833, -52.051)> <SkyCoord (ICRS): (ra, dec) in deg
    (39.21541667, -52.05111111)>
<SkyCoord (ICRS): (ra, dec) in deg
    (39.21545833, -52.051)> <SkyCoord (ICRS): (ra, dec) in deg
    (39.215423, -52.0510003)>
<SkyCoord (ICRS): (ra, dec) in deg
    (39.21545833, -52.051)> <SkyCoord (ICRS): (ra, dec) in deg
    (39.21545833, -52.051)>
<SkyCoord (ICRS): (ra, dec) in deg
    (42.74829167, -34.15138889)> <SkyCoord (I

<SkyCoord (ICRS): (ra, dec) in deg
    (72.00275, -50.6904167)> <SkyCoord (ICRS): (ra, dec) in deg
    (72.00275, -50.69041667)>
<SkyCoord (ICRS): (ra, dec) in deg
    (351.5445417, -73.3971667)> <SkyCoord (ICRS): (ra, dec) in deg
    (351.54458333, -73.39722222)>
<SkyCoord (ICRS): (ra, dec) in deg
    (351.5445417, -73.3971667)> <SkyCoord (ICRS): (ra, dec) in deg
    (351.54454167, -73.39716667)>
<SkyCoord (ICRS): (ra, dec) in deg
    (7.6071667, -62.6004167)> <SkyCoord (ICRS): (ra, dec) in deg
    (7.60717, -62.6004)>
<SkyCoord (ICRS): (ra, dec) in deg
    (31.7573333, -44.1105556)> <SkyCoord (ICRS): (ra, dec) in deg
    (31.75733333, -44.11055556)>
<SkyCoord (ICRS): (ra, dec) in deg
    (33.2424583, -58.8550556)> <SkyCoord (ICRS): (ra, dec) in deg
    (33.2428333, -58.8550833)>
<SkyCoord (ICRS): (ra, dec) in deg
    (33.8886667, -56.4548611)> <SkyCoord (ICRS): (ra, dec) in deg
    (33.8890417, -56.4548889)>
<SkyCoord (ICRS): (ra, dec) in deg
    (35.214125, -58.39475)> <SkyCoord (IC

<SkyCoord (ICRS): (ra, dec) in deg
    (172.98041667, -34.6075)> <SkyCoord (ICRS): (ra, dec) in deg
    (172.98025, -34.60755556)>
<SkyCoord (ICRS): (ra, dec) in deg
    (177.10125, -37.48027778)> <SkyCoord (ICRS): (ra, dec) in deg
    (177.1009279, -37.4803089)>
<SkyCoord (ICRS): (ra, dec) in deg
    (177.10125, -37.48027778)> <SkyCoord (ICRS): (ra, dec) in deg
    (177.10091667, -37.48030556)>
<SkyCoord (ICRS): (ra, dec) in deg
    (347.97333333, -45.13333333)> <SkyCoord (ICRS): (ra, dec) in deg
    (347.97341667, -45.13344444)>
<SkyCoord (ICRS): (ra, dec) in deg
    (170.27291667, -38.75444444)> <SkyCoord (ICRS): (ra, dec) in deg
    (170.2728384, -38.7545893)>
<SkyCoord (ICRS): (ra, dec) in deg
    (314.0114125, -17.1816122)> <SkyCoord (ICRS): (ra, dec) in deg
    (314.01141667, -17.18161111)>
<SkyCoord (ICRS): (ra, dec) in deg
    (131.9864875, -78.9147747)> <SkyCoord (ICRS): (ra, dec) in deg
    (131.9865, -78.91477778)>
<SkyCoord (ICRS): (ra, dec) in deg
    (165.4662726, -34.70

# Save final sample

In [45]:
fullsample.write('Catalogs/mactivity_movinggroups_confirmed.fits',format='fits',
                  overwrite=True)